In [33]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define the function to compute MAE
def compute_mae(image1, image2):
    return np.abs(image1 - image2).mean()

# Paths to the folders
gt_folder = "/home/PET-CT/splited_data_15k/test/B"
condition_folder = "/home/PET-CT/splited_data_15k/test/A"
# pre_folder = "results/108_CT2PET_UncerBBDM3c/LBBDM-f4/sample_to_eval/200"

dataset_name = 'UncerBBDM_2Unet_confloss_15k_training_samples_top_model'

pre_folder = "/home/PET-CT/thaind/BBDM_folk/results/" + dataset_name + "/LBBDM-f4/sample_to_eval/200"




# Lists to store the computed metrics for each pair
# ssim_scores = []
# psnr_scores = []

mae_scores = []
high_mae, high_mae_gts, high_mae_pds, high_mae_conditions = [], [], [], []

# Iterate through the files in the ground truth folder
for filename in os.listdir(gt_folder):
    # Make sure the file is a numpy array
    if filename.endswith(".npy"):
        # Construct the paths for the corresponding ground truth and predicted files
        try:
            gt_path = os.path.join(gt_folder, filename)
            pre_path = os.path.join(pre_folder, filename)
        
            # Load the images as numpy arrays
            gt_img = np.load(gt_path, allow_pickle=True)
            pre_img = np.load(pre_path, allow_pickle=True)
        except:
            continue   
        # Preprocess the predicted image
        pre_img1 = pre_img.mean(axis=-1) / 32767.0
        
        # Normalize the ground truth image
        gt_img1 = gt_img / 32767.0
        # Calculate the SSIM, PSNR, and MAE for this pair
        # ssim_score = ssim(pre_img, gt_img, data_range=1)
        # psnr_score = psnr(pre_img, gt_img, data_range=1)
        mae = compute_mae(pre_img1, gt_img1)
    
        # Append the scores to the corresponding lists
        # ssim_scores.append(ssim_score)
        # psnr_scores.append(psnr_score)
        mae_scores.append(mae * 32767)

        if mae * 32767 > 1000  : 
            high_mae_gts.append(gt_img) 
            high_mae_pds.append(pre_img)
            high_mae_conditions.append(np.load(os.path.join(condition_folder, filename), allow_pickle=True))
            high_mae.append(mae * 32767)

# Calculate the mean scores over all pairs
# mean_ssim = np.mean(ssim_scores)
# mean_psnr = np.mean(psnr_scores)
mean_mae = np.mean(mae_scores)

# Print the mean metrics
# print("Mean SSIM: {}".format(mean_ssim))
# print("Mean PSNR: {}".format(mean_psnr))
print("Mean MAE: {}".format(mean_mae))


Mean MAE: 290.6669934240417


In [34]:
import matplotlib.pyplot as plt

save_fig_dir = './fig/' + dataset_name
# save_fig_dir = './fig/small_mae/' + dataset_name

if not os.path.exists(save_fig_dir):
    os.makedirs(save_fig_dir)


# Define a function to visualize images
def visualize_and_save(gt, pre, condition , mae, filename):
    plt.figure(figsize=(18, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(condition, cmap='gray')
    plt.title('CT')
    
    plt.subplot(1, 3, 2)
    plt.imshow(gt, cmap='gray')
    plt.title('Ground Truth')

    plt.subplot(1, 3, 3)
    plt.imshow(pre, cmap='gray')
    plt.title('Predicted')

    
    plt.suptitle(f'MAE: {mae:.2f}')
    plt.savefig(filename)
    plt.close()

# Visualize images with high MAE



for i in range(len(high_mae)):
    gt_image = high_mae_gts[i]
    pre_image = high_mae_pds[i]
    mae_value = high_mae[i]
    condition_img = high_mae_conditions[i]
    filename = os.path.join(save_fig_dir, str(i) + '.png' )
    # title = f"MAE: {mae_value:.2f}"
    visualize_and_save(gt_image, pre_image, condition_img, mae_value, filename)


In [10]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define the function to compute MAE
def compute_mae(image1, image2):
    return np.abs(image1 - image2).mean()

# Paths to the folders
gt_folder = "/home/PET-CT/splited_data_15k/test/B"
# pre_folder = "results/108_CT2PET_UncerBBDM3c/LBBDM-f4/sample_to_eval/200"
pre_folder = "/home/PET-CT/thaind/BBDM_folk/results/UncerBBDM_4Unet_confloss_15k_training_samples_latest_model/LBBDM-f4/sample_to_eval/200"

# Lists to store the computed metrics for each pair
ssim_scores = []
psnr_scores = []
mae_scores = []

# Iterate through the files in the ground truth folder
for filename in os.listdir(gt_folder):
    # Make sure the file is a numpy array
    if filename.endswith(".npy"):
        # Construct the paths for the corresponding ground truth and predicted files
        try:
            gt_path = os.path.join(gt_folder, filename)
            pre_path = os.path.join(pre_folder, filename)
        
            # Load the images as numpy arrays
            gt_img = np.load(gt_path, allow_pickle=True)
            pre_img = np.load(pre_path, allow_pickle=True)
        except:
            continue   
        # Preprocess the predicted image
        pre_img = pre_img.mean(axis=-1) / 32767.0
        
        # Normalize the ground truth image
        gt_img = gt_img / 32767.0
        
        # Calculate the SSIM, PSNR, and MAE for this pair
        ssim_score = ssim(pre_img, gt_img, data_range=1)
        psnr_score = psnr(pre_img, gt_img, data_range=1)
        mae = compute_mae(pre_img, gt_img)
        
        # Append the scores to the corresponding lists
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
        mae_scores.append(mae * 32767)

# Calculate the mean scores over all pairs
mean_ssim = np.mean(ssim_scores)
mean_psnr = np.mean(psnr_scores)
mean_mae = np.mean(mae_scores)

# Print the mean metrics
print("Mean SSIM: {}".format(mean_ssim))
print("Mean PSNR: {}".format(mean_psnr))
print("Mean MAE: {}".format(mean_mae))


Mean SSIM: 0.9568914975272752
Mean PSNR: 32.90935576932342
Mean MAE: 206.8017248553323
